# Voltron

This notebook predict the health or not of Grape.

## What is the SVM algorithm?

 It is a supervised learning algorithm designed to solve discrimination and regression problems.
 
 It is a very good image classification algorithm.
 
 ![enter image description here](https://editor.analyticsvidhya.com/uploads/61706svm3.png)
 
 ## Summary
 
 1. [Data preparation](#prepaData)
 2. [Learning of model](#model)
 3. [Displaying metrcis](#metric)
 4. [Conclusion](#conclusion)

### 1. Data preparation <a id="prepaData"></a>
 
 - To start, we import the necessary libraries.

In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import cv2
from tqdm import tqdm

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 - We define all necessary variables.

In [17]:
path = 'Grape/'
IMG_SIZE = 100

- We create the function that load all our data and we call it

In [18]:
def loadingData(path):
    CATEGORIES = []
    data = []
    
    dirs = os.listdir(path)
    for file in dirs:
        CATEGORIES.append(file)

    for category in tqdm(CATEGORIES):
        localPath = os.path.join(path, category)
        for img in os.listdir(localPath):
            img_array = cv2.imread(os.path.join(localPath, img))
            new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
            data.append([new_array, category])
    
    return data

In [32]:
data = loadingData(path)

100%|██████████| 4/4 [00:01<00:00,  2.54it/s]


- We create the function that preprocess all our data and we call it

In [20]:
from sklearn.model_selection import train_test_split

def preprocessing(df):
    x = []
    y = []

    for categories, label in df:
        x.append(categories)
        y.append(label)
        
    x = np.array(x).reshape(len(x),-1)
    x = x / 255.0
        
    #split
    return train_test_split(x, y, test_size=0.33, random_state=42)

In [34]:
X_train, x_test, y_train, y_test = preprocessing(data)
print(X_train.shape)
print(x_test.shape)

(1144, 30000)
(564, 30000)


### 2. Learning of model <a id="model"></a>

- We find the optimal parameter

In [22]:
from sklearn.svm import SVC
# from sklearn.model_selection import GridSearchCV

# param_grid = {'C': [0.1, 1, 10, 100, 1000],
#               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
#               'kernel': ['rbf', 'linear']}

# grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
 
# # fitting the model for grid search
# grid.fit(X_train, y_train)

In [23]:
# # print best parameter after tuning
# print(grid.best_params_)
 
# # print how our model looks after hyper-parameter tuning
# print(grid.best_estimator_)

- We start our model learning

In [24]:
# svc = SVC(kernel='rbf',gamma=grid.best_params_['gamma'], C=grid.best_params_['C'], class_weight='balanced')
svc = SVC(kernel='rbf',gamma=0.001, C=10, class_weight='balanced')
svc.fit(X_train, y_train)

SVC(C=10, class_weight='balanced', gamma=0.001)

In [25]:
pred = svc.predict(x_test)

### 2. Displaying metrcis<a id="metric"></a>
- Accuracy

In [26]:
from sklearn.metrics import accuracy_score
print("Accuracy on data is",accuracy_score(y_test,pred))

Accuracy on data is 0.8670212765957447


The precision is about 86 %

In [27]:
from sklearn.metrics import classification_report, confusion_matrix
print("Classification report is")
print(classification_report(y_test,pred))

Classification report is
               precision    recall  f1-score   support

Black_Measles       0.80      0.85      0.83       131
    Black_rot       0.81      0.73      0.77       151
      Healthy       0.94      0.98      0.96       133
  Leaf_blight       0.90      0.92      0.91       149

     accuracy                           0.87       564
    macro avg       0.87      0.87      0.87       564
 weighted avg       0.87      0.87      0.87       564



We see that our model are more difficulty to recognize black rot ( but its very correct).

In [28]:
print('confusion matrix')
print(confusion_matrix(y_test, pred))

confusion matrix
[[111  15   1   4]
 [ 25 110   5  11]
 [  0   2 131   0]
 [  2   8   2 137]]


The confusion matrix looks good but confirm the classification report

In [29]:
result = pd.DataFrame({'original' : y_test,'predicted' : pred})
result[:5]

,original,predicted
0,Black_rot,Leaf_blight
1,Leaf_blight,Leaf_blight
2,Leaf_blight,Leaf_blight
3,Black_Measles,Black_Measles
4,Black_rot,Black_rot


In [30]:
import pickle
filename = 'svm_model.sav'
pickle.dump(svc, open(filename, 'wb'))

### 2. Conclusion<a id="conclusion"></a>

Our overall accuracy is not too bad with 86% accuracy. 